### Imports

In [1]:
import pandas as pd
import numpy as np
import pymongo
import json
import cx_Oracle

# path da instalação do instantclient
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient_21_9")

### Estabelecer conexão

In [2]:
# Conectar ao Oracle
dsn_tns = cx_Oracle.makedsn('localhost', '1521', service_name='xe')
conn = cx_Oracle.connect(user='store', password='uminho2023', dsn=dsn_tns)

### Selecionar dados das tabelas

In [3]:
oracle_cursor = conn.cursor()

----
Grupo 1: USER

STORE_USERS

In [4]:
oracle_cursor.execute("SELECT * FROM STORE_USERS")
results_storeUsers = oracle_cursor.fetchall()

store_users = pd.DataFrame(results_storeUsers, columns=[i[0] for i in oracle_cursor.description])

ADDRESSES

In [5]:
oracle_cursor.execute("SELECT * FROM ADDRESSES")
results_addresses = oracle_cursor.fetchall()

addresses = pd.DataFrame(results_addresses, columns=[i[0] for i in oracle_cursor.description])

----
Grupo 2: EMPLOYEES

EMPLOYEES

In [6]:
oracle_cursor.execute("SELECT * FROM EMPLOYEES")
results_employees = oracle_cursor.fetchall()

employees = pd.DataFrame(results_employees, columns=[i[0] for i in oracle_cursor.description])

DEPARTMENTS

In [7]:
oracle_cursor.execute("SELECT * FROM DEPARTMENTS")
results_departaments = oracle_cursor.fetchall()

departaments = pd.DataFrame(results_departaments, columns=[i[0] for i in oracle_cursor.description])

EMPLOYEES_ARCHIVE

In [8]:
oracle_cursor.execute("SELECT * FROM EMPLOYEES_ARCHIVE")
results_employeesArchive = oracle_cursor.fetchall()

employeesArchive = pd.DataFrame(results_employeesArchive, columns=[i[0] for i in oracle_cursor.description])

----
Grupo 3: PRODUCT  

PRODUCT

In [9]:
oracle_cursor.execute("SELECT * FROM PRODUCT")
results_product = oracle_cursor.fetchall()

product = pd.DataFrame(results_product, columns=[i[0] for i in oracle_cursor.description])

PRODUCT_CATEGORIES

In [10]:
oracle_cursor.execute("SELECT * FROM PRODUCT_CATEGORIES")
results_productCategories = oracle_cursor.fetchall()

productCategories = pd.DataFrame(results_productCategories, columns=[i[0] for i in oracle_cursor.description])

DISCOUNT

In [11]:
oracle_cursor.execute("SELECT * FROM DISCOUNT")
results_discount = oracle_cursor.fetchall()

discount = pd.DataFrame(results_discount, columns=[i[0] for i in oracle_cursor.description])

STOCK

In [12]:
oracle_cursor.execute("SELECT * FROM STOCK")
results_stock = oracle_cursor.fetchall()

stock = pd.DataFrame(results_stock, columns=[i[0] for i in oracle_cursor.description])

----
Grupo 4: ORDER  
  
  
PAYMENT_DETAILS

In [13]:
oracle_cursor.execute("SELECT * FROM PAYMENT_DETAILS")
results_paymentDetails = oracle_cursor.fetchall()

payment_details = pd.DataFrame(results_paymentDetails, columns=[i[0] for i in oracle_cursor.description])

ORDER_DETAILS

In [14]:
oracle_cursor.execute("SELECT * FROM ORDER_DETAILS")
results_orderDetails = oracle_cursor.fetchall()

order_details = pd.DataFrame(results_orderDetails, columns=[i[0] for i in oracle_cursor.description])

ORDER_ITEMS

In [15]:
oracle_cursor.execute("SELECT * FROM ORDER_ITEMS")
results_orderItems = oracle_cursor.fetchall()

order_items = pd.DataFrame(results_orderItems, columns=[i[0] for i in oracle_cursor.description])

CART_ITEM

In [16]:
oracle_cursor.execute("SELECT * FROM CART_ITEM")
results_cartItem = oracle_cursor.fetchall()

cart_item = pd.DataFrame(results_cartItem, columns=[i[0] for i in oracle_cursor.description])

SHOPPING_SESSION

In [17]:
oracle_cursor.execute("SELECT * FROM SHOPPING_SESSION")
results_shoppingSession = oracle_cursor.fetchall()

shopping_session = pd.DataFrame(results_shoppingSession, columns=[i[0] for i in oracle_cursor.description])

----
# Tratar os dados

Temos de agrupar os dados das tabelas tal como planeado.  
    - Juntar em 4 grupos  
    - Limpar nulls  
    - etc  

### EMPLOYEES
- Passar a ter um único documento para employees
- Adicionar um campo de "estado" (antigo/atual)


In [18]:
employee_state = "" # (active, old)

employee_doc = {
    "emp_ID": employees['EMPLOYEE_ID'],
    "emp_firstName": employees['FIRST_NAME'],
    "emp_MiddleName": employees['MIDDLE_NAME'],
    "emp_LastName": employees['LAST_NAME'],
    "emp_dateOfBirth": employees['DATE_OF_BIRTH'],
    "emp_departmentID": employees['DEPARTMENT_ID'],
    "emp_hireDate": employees['HIRE_DATE'],
    "emp_salary": employees['SALARY'],
    "emp_phoneNumber": employees['PHONE_NUMBER'],
    "emp_Email": employees['EMAIL'],
    "emp_ssnNumber": employees['SSN_NUMBER'],
    "emp_managerID": employees['MANAGER_ID'],
    "employee_state": employee_state
}

# Mais à frente temos de atualizar o campo `employee_state` com o estado certinho

# Pode ser aplicado aqui um trigger para atualizar `employee_state` na BD quando for alterado.

In [19]:
# Iterar pelo EMPLOYEES_ARCHIVE e descontruir em 2 registos (um com o estado "old", e o outro com o estado "active")

old_employees = []
active_employees = []

for index, row in employeesArchive.iterrows():
    
    if row['OLD_EMPLOYEE_ID'] is not None and not np.isnan(row['OLD_EMPLOYEE_ID']):
        employee_info_old = {
        "emp_ID": row['OLD_EMPLOYEE_ID'],
        "emp_firstName": row['OLD_FIRST_NAME'],
        "emp_MiddleName": row['OLD_MIDDLE_NAME'],
        "emp_LastName": row['OLD_LAST_NAME'],
        "emp_dateOfBirth": str(row['OLD_DATE_OF_BIRTH']),
        "emp_departmentID": row['OLD_DEPARTMENT_ID'],
        "emp_hireDate": str(row['OLD_HIRE_DATE']),
        "emp_salary": row['OLD_SALARY'],
        "emp_phoneNumber": row['OLD_PHONE_NUMBER'],
        "emp_Email": row['OLD_EMAIL'],
        "emp_ssnNumber": row['OLD_SSN_NUMBER'],
        "emp_managerID": row['OLD_MANAGER_ID'],
        "employee_state": "old"
        }
        old_employees.append(employee_info_old)

    employee_info_active = {
            "emp_ID": row['NEW_EMPLOYEE_ID'],
            "emp_firstName": row['NEW_FIRST_NAME'],
            "emp_MiddleName": row['NEW_MIDDLE_NAME'],
            "emp_LastName": row['NEW_LAST_NAME'],
            "emp_dateOfBirth": str(row['NEW_DATE_OF_BIRTH']),
            "emp_departmentID": row['NEW_DEPARTMENT_ID'],
            "emp_hireDate": str(row['NEW_HIRE_DATE']),
            "emp_salary": row['NEW_SALARY'],
            "emp_phoneNumber": row['NEW_PHONE_NUMBER'],
            "emp_Email": row['NEW_EMAIL'],
            "emp_ssnNumber": row['NEW_SSN_NUMBER'],
            "emp_managerID": row['NEW_MANAGER_ID'],
            "employee_state": "active"
    }
    active_employees.append(employee_info_active)


old_json_object = json.dumps(old_employees, indent=4)
active_json_object = json.dumps(active_employees, indent=4)

#print("Old employees:")
#print(old_json_object)
#print("Active employees:")
#print(active_json_object)

----
# Enviar para o Mongo

In [21]:

""" 
import pymongo

# Conectar ao MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Criar uma base de dados chamada "oes"
database = client["oes"]

# Criar as coleções
collection_user = database["user"]
collection_product = database["product"]
collection_order = database["order"]
collection_employees = database["employees"]


# Converter o dataframe numa lista de dicionários
documents = df_merged.to_dict(orient='records')

# Inserir os documentos na coleção
collection.insert_many(documents)
"""

' \nimport pymongo\n\n# Conectar ao MongoDB\nclient = pymongo.MongoClient("mongodb://localhost:27017/")\n\n# Criar uma base de dados chamada "oes"\ndatabase = client["oes"]\n\n# Criar as coleções\ncollection_user = database["user"]\ncollection_product = database["product"]\ncollection_order = database["order"]\ncollection_employees = database["employees"]\n\n\n# Converter o dataframe numa lista de dicionários\ndocuments = df_merged.to_dict(orient=\'records\')\n\n# Inserir os documentos na coleção\ncollection.insert_many(documents)\n'